In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

pd.set_option('display.max_columns',999)

In [31]:
# temp_product = product[['CLNT_ID','SESS_ID','PD_C']]
# temp_product.drop_duplicates(inplace = True)

# temp_merge = pd.merge(temp_product, master[['PD_C','CLAC1_NM']], how = 'left')
# temp_merge = temp_merge[['CLNT_ID','SESS_ID','CLAC1_NM']]
# temp_merge.drop_duplicates(inplace = True)

# temp_count = temp_merge.groupby(['CLNT_ID','SESS_ID']).count().reset_index()

# association_df = pd.merge(temp_count[lambda x : x.CLAC1_NM > 1][['CLNT_ID','SESS_ID']], temp_merge, how = 'left', on = ['CLNT_ID','SESS_ID'])

# association_dummydf = pd.get_dummies(association_df, prefix = 'CLAC1_NM').groupby(['CLNT_ID','SESS_ID']).sum()
# association_dummydf.to_csv('data/association_dummydf.csv',  index = False)

### 상품군을 동시에 1개 이상 구매한 경우 : df 만들기

# 연관규칙 분석

<연관성 규칙 분석>
- 특정 제품, 사건들이 동시에 발생하는 패턴을 파악하는데 이용
- 단, 인과관계는 아님

< 적용 분야 >
- 교차 판매, 상품진열, 보험 부정행위 적발 등등

< 연관성 규칙 결과 유형 >
- useful result : 마케팅 전략상 유용한 결과가 나온 경우
- triivalresult : 기존의 마케팅 전략에 의해 연관성이 높게 나온 경우
- inexplicable result : 의미를 발견하기 위해 많은 고민이 필요한 경우

< 연관성규칙 생성 과정 >
- 최소지지도를 기준으로 상품묶음 명단을 정하고, 명단 모두 신뢰도를 계산하여 어떤 놈이 선행이 되는 것인지 판단하여 규칙을 생성하고 강도 판단, 향상도를 통해 해당 규칙 판단

1단계 : 지지도
- 빈발항목집합들(frequent item sets)에 기초하여 후보규칙들의 집합결정
- 대표 알고리즘 : Apriori
- 전체 구매중 얘가 구매에 포함되어 있을 확률
    
2단계 : 신뢰도
- 연관 규칙의 강도
- A를 구매한 경우, 이 중에서 얼마나 항목 B를 구매할 것인지를 의미
(지지도는 같이 사는 놈들을 걸러준다면 신뢰도는 같이 살 때 어떤 놈이 선행되서 사는 것인지 판단)

3단계 : 향상도
- 향상도 > 1 : 두 품목이 서로 양의 상관관계
- 향상도 < 1 : 두 품목이 서로 음의 상관관계
- 향상도 = 1 : 두 품목이 서로 독립
<br>

< 확률모형 >
A->B 를 if A then B라는 연관규칙이라고 하면,
- 지지도(support) s(A→B) 
= A와 B를 동시에 포함하는 거래 수 / 전체 거래 수  = P(A∩B)
- 신뢰도(Confidence) c(A→B) 
= A와 B를 동시에 포함하는 거래 수 / A가 포함된 거래 수 =  P(A∩B) / P(A) = 지지도/P(A)
- 향상도(Lift)
= A와 B를 동시에 포함할 확률 / A의 확률 * B의 확률 = P(A∩B) / P(A)*P(B)  = P (B|A) / P (B) = 신뢰도 / P(B)

<예시> 
맥주 -> 콜라 : 지지도 = 0.5, 신뢰도 = 1, 향상도 = 1.5
- 지지도 : 맥주와 콜라를 동시에 구매한 사람은 전체구매 중 0.5
- 신뢰도 : 맥주를 구매한 사람들은 모두 콜라도 같이 구매
- 향상도 : 맥주 구매시 콜라를 구입하게 될 가능성은 그냥 콜라를 구매할 때 보다 1.5배 높아짐

<순차패턴분석>
- 연관규칙에 시간의 변이를 추가한 방법
- 주가변동패턴, 구매패턴

In [3]:
association_dummydf = pd.read_csv('data/association_dummydf.csv', encoding = 'euc-kr')

In [4]:
df = association_dummydf.copy()

In [5]:
import pandas as pd 
import itertools 

def support(df, item_lst):
    return (df[list(item_lst)].sum(axis=1)==len(item_lst)).mean()

def make_all_set_over_support(df, support_threshold):
    items = []
    single_items = [col for col in df.columns if support(df, [col]) > support_threshold] # size 1 items
    
    size = 2
    while True:
        new_items = []
        for item_cand in itertools.combinations(single_items, size):
            #print(item_cand, (df[list(item_cand)].sum(axis=1)==size).mean())
            if support(df, list(item_cand)) > support_threshold:
                new_items.append(list(item_cand))
        if len(new_items)==0:
            break
        else:
            items+=new_items
            size+=1
    items += [ [s] for s in single_items]# 이렇게 해줘야 모든 type이 list가 됨
    return items

def make_confidence_lst(df, item_set_over_support, confidence_threshold):
    r_lst = []
    for item1 in item_set_over_support:
        for item2 in item_set_over_support:
            if len(set(item1).intersection(set(item2)))==0:
                conf = support(df, list(set(item1).union(set(item2))))/ support(df, item1)
                if conf > confidence_threshold:
                    r_lst.append((item1, item2, conf))
            else:
                continue
    return sorted(r_lst, key=lambda x: x[2], reverse=True)

def make_lift_lst(df, item_set_over_support, lift_threhsold):
    r_lst = []
    for item1 in item_set_over_support:
        for item2 in item_set_over_support:
            if len(set(item1).intersection(set(item2)))==0:
                lift = support(df, list(set(item1).union(set(item2))))
                lift /= support(df, item1)
                lift /= support(df, item2)
                if lift > lift_threhsold:
                    r_lst.append((item1, item2, lift))
            else:
                continue
    return sorted(r_lst, key=lambda x: x[2], reverse=True)

In [30]:
len(df) / 100

3504.36

In [26]:
over_support_lst = make_all_set_over_support(df, 0.01)
print(over_support_lst)

[['CLAC1_NM_건강식품', 'CLAC1_NM_화장품/뷰티케어'], ['CLAC1_NM_구기/필드스포츠', 'CLAC1_NM_스포츠패션'], ['CLAC1_NM_남성의류', 'CLAC1_NM_속옷/양말/홈웨어'], ['CLAC1_NM_남성의류', 'CLAC1_NM_스포츠패션'], ['CLAC1_NM_남성의류', 'CLAC1_NM_시즌스포츠'], ['CLAC1_NM_남성의류', 'CLAC1_NM_여성의류'], ['CLAC1_NM_남성의류', 'CLAC1_NM_유아동의류'], ['CLAC1_NM_남성의류', 'CLAC1_NM_패션잡화'], ['CLAC1_NM_남성의류', 'CLAC1_NM_화장품/뷰티케어'], ['CLAC1_NM_속옷/양말/홈웨어', 'CLAC1_NM_스포츠패션'], ['CLAC1_NM_속옷/양말/홈웨어', 'CLAC1_NM_여성의류'], ['CLAC1_NM_속옷/양말/홈웨어', 'CLAC1_NM_유아동의류'], ['CLAC1_NM_속옷/양말/홈웨어', 'CLAC1_NM_출산/육아용품'], ['CLAC1_NM_속옷/양말/홈웨어', 'CLAC1_NM_패션잡화'], ['CLAC1_NM_속옷/양말/홈웨어', 'CLAC1_NM_화장품/뷰티케어'], ['CLAC1_NM_스포츠패션', 'CLAC1_NM_시즌스포츠'], ['CLAC1_NM_스포츠패션', 'CLAC1_NM_아웃도어/레저'], ['CLAC1_NM_스포츠패션', 'CLAC1_NM_여성의류'], ['CLAC1_NM_스포츠패션', 'CLAC1_NM_유아동의류'], ['CLAC1_NM_스포츠패션', 'CLAC1_NM_패션잡화'], ['CLAC1_NM_스포츠패션', 'CLAC1_NM_화장품/뷰티케어'], ['CLAC1_NM_시즌스포츠', 'CLAC1_NM_여성의류'], ['CLAC1_NM_시즌스포츠', 'CLAC1_NM_유아동의류'], ['CLAC1_NM_시즌스포츠', 'CLAC1_NM_패션잡화'], ['CLAC1_NM_식기/조리기구', 'CLAC1_NM_주방잡화'], ['CLAC1_NM_여성의류',

In [27]:
print("over confidence list")
for a, b, conf in  make_confidence_lst(df, over_support_lst, 0.2):
    print("{} => {}: {}".format(a, b, conf))

over confidence list
['CLAC1_NM_구기/필드스포츠'] => ['CLAC1_NM_스포츠패션']: 0.5400661409310608
['CLAC1_NM_퍼스널케어'] => ['CLAC1_NM_화장품/뷰티케어']: 0.4706255091880148
['CLAC1_NM_아웃도어/레저'] => ['CLAC1_NM_스포츠패션']: 0.4343526087395297
['CLAC1_NM_헬스/피트니스'] => ['CLAC1_NM_스포츠패션']: 0.38895582329317263
['CLAC1_NM_남성의류'] => ['CLAC1_NM_여성의류']: 0.38091041845007306
['CLAC1_NM_여성의류'] => ['CLAC1_NM_남성의류']: 0.3229828250589755
['CLAC1_NM_주방잡화'] => ['CLAC1_NM_식기/조리기구']: 0.30517068990144264
['CLAC1_NM_시즌스포츠'] => ['CLAC1_NM_스포츠패션']: 0.28740172433438416
['CLAC1_NM_식기/조리기구'] => ['CLAC1_NM_주방잡화']: 0.2812109246462652
['CLAC1_NM_세제/위생'] => ['CLAC1_NM_출산/육아용품']: 0.27228721258572003
['CLAC1_NM_남성의류'] => ['CLAC1_NM_스포츠패션']: 0.26606324588347857
['CLAC1_NM_유아동의류'] => ['CLAC1_NM_스포츠패션']: 0.2572892636992777
['CLAC1_NM_시즌스포츠'] => ['CLAC1_NM_패션잡화']: 0.2542063883930267
['CLAC1_NM_여성의류', 'CLAC1_NM_퍼스널케어'] => ['CLAC1_NM_화장품/뷰티케어']: 0.24615384615384617
['CLAC1_NM_유아동의류'] => ['CLAC1_NM_속옷/양말/홈웨어']: 0.2459819233823696
['CLAC1_NM_패션잡화'] => ['CL

In [28]:
print("over lift list")
for a, b, lift in  make_lift_lst(df, over_support_lst, 1):
    print("{} => {}: {}".format(a, b, lift))

over lift list
['CLAC1_NM_모바일'] => ['CLAC1_NM_컴퓨터']: 10.084226298226007
['CLAC1_NM_컴퓨터'] => ['CLAC1_NM_모바일']: 10.084226298226007
['CLAC1_NM_식기/조리기구'] => ['CLAC1_NM_주방잡화']: 7.038025395610526
['CLAC1_NM_주방잡화'] => ['CLAC1_NM_식기/조리기구']: 7.038025395610526
['CLAC1_NM_가구'] => ['CLAC1_NM_청소/세탁/욕실용품']: 5.313282044767689
['CLAC1_NM_청소/세탁/욕실용품'] => ['CLAC1_NM_가구']: 5.313282044767688
['CLAC1_NM_주방잡화'] => ['CLAC1_NM_청소/세탁/욕실용품']: 4.490026606164778
['CLAC1_NM_청소/세탁/욕실용품'] => ['CLAC1_NM_주방잡화']: 4.490026606164778
['CLAC1_NM_건강식품'] => ['CLAC1_NM_음료']: 4.218544765814356
['CLAC1_NM_음료'] => ['CLAC1_NM_건강식품']: 4.218544765814356
['CLAC1_NM_모바일'] => ['CLAC1_NM_문구/사무용품']: 3.89779061975316
['CLAC1_NM_문구/사무용품'] => ['CLAC1_NM_모바일']: 3.89779061975316
['CLAC1_NM_출산/육아용품'] => ['CLAC1_NM_세제/위생']: 3.7222251464673057
['CLAC1_NM_세제/위생'] => ['CLAC1_NM_출산/육아용품']: 3.7222251464673053
['CLAC1_NM_과일'] => ['CLAC1_NM_건강식품']: 3.632512494295128
['CLAC1_NM_건강식품'] => ['CLAC1_NM_과일']: 3.6325124942951277
['CLAC1_NM_문구/사무용품'] => ['CL

In [31]:
df.columns

Index(['CLAC1_NM_가구', 'CLAC1_NM_건강식품', 'CLAC1_NM_계절가전', 'CLAC1_NM_과일',
       'CLAC1_NM_구기/필드스포츠', 'CLAC1_NM_남성의류', 'CLAC1_NM_냉동식품',
       'CLAC1_NM_냉장/세탁가전', 'CLAC1_NM_냉장식품', 'CLAC1_NM_모바일', 'CLAC1_NM_문구/사무용품',
       'CLAC1_NM_상품권', 'CLAC1_NM_생활/주방가전', 'CLAC1_NM_세제/위생',
       'CLAC1_NM_속옷/양말/홈웨어', 'CLAC1_NM_스포츠패션', 'CLAC1_NM_시즌스포츠',
       'CLAC1_NM_식기/조리기구', 'CLAC1_NM_아웃도어/레저', 'CLAC1_NM_여성의류',
       'CLAC1_NM_영상/음향가전', 'CLAC1_NM_완구', 'CLAC1_NM_원예/애완', 'CLAC1_NM_유아동의류',
       'CLAC1_NM_음료', 'CLAC1_NM_인테리어/조명', 'CLAC1_NM_자동차용품', 'CLAC1_NM_주방잡화',
       'CLAC1_NM_청소/세탁/욕실용품', 'CLAC1_NM_축산물', 'CLAC1_NM_출산/육아용품',
       'CLAC1_NM_침구/수예', 'CLAC1_NM_컴퓨터', 'CLAC1_NM_패션잡화', 'CLAC1_NM_퍼스널케어',
       'CLAC1_NM_헬스/피트니스', 'CLAC1_NM_화장품/뷰티케어'],
      dtype='object')